In [2]:
from pyscf import gto, dft, scf, lib, cc, fci
import numpy as np
import scipy as sp

Numpy 1.16 has memory leak bug  https://github.com/numpy/numpy/issues/13808
It is recommended to downgrade to numpy 1.15 or older


# 1) Definitions

In [4]:
S_A = gto.Mole(atom="O 0 0 -0.01", basis="sto3g",charge=0,spin=0)
S_A.build()

In [5]:
S_B = gto.Mole(atom="C 0 0 0; O 0 0 0.01 ", basis="sto3g",charge=0, spin=0)
S_B.build()

In [6]:
S_T = gto.Mole(atom=S_A.atom + "; " +S_B.atom,
               basis="sto3g",
               charge=S_A.charge+S_B.charge,
               spin=S_A.spin+S_B.spin)

S_T.build()

In [5]:
# grids = dft.gen_grid.Grids(S_T)
# grids.level = 4
# grids.build()

In [7]:
na = S_A.nao_nr()
nb = S_B.nao_nr()
nt = S_T.nao_nr()

na,nb,nt

(5, 10, 15)

In [8]:
S_Tscf = scf.RKS(S_T)
S_Tscf.xc = 'lda,vwn'
S_Tscf.small_rho_cutoff = 1e-20
S_Tscf.kernel()

/Users/lex/anaconda3/envs/pyscf/lib/python3.7/site-packages/pyscf/scf/chkfile.py:31: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(chkfile) as fh5:


SCF not converged.
SCF energy = 6145.50457447892


6145.50457447892

# 2 ) Compute total Fock Matrix, Orbital Overlap Matrix and make guess on initial Density Matrices

In [9]:
# Fock_T = S_Tscf.get_hcore() + S_Tscf.get_veff()

dm_T = S_Tscf.make_rdm1()
J_Tot, K_Tot = S_Tscf.get_jk(dm=dm_T) # Coulomb (J) and exchange (K)
Fock_T = S_Tscf.get_hcore() + J_Tot + K_Tot

In [10]:
M_overlaps = S_Tscf.get_ovlp()

In [12]:
S_Ascf = scf.RKS(S_A)
S_Ascf.xc = 'lda,vwn'
S_Ascf.small_rho_cutoff = 1e-20
S_Ascf.kernel()

converged SCF energy = -73.3282156705682


-73.3282156705682

In [13]:
S_Bscf = scf.RKS(S_B)
S_Bscf.xc = 'lda,vwn'
S_Bscf.small_rho_cutoff = 1e-20
S_Bscf.kernel()

SCF not converged.
SCF energy = 2289.31624570067


2289.316245700671

In [14]:
Dmat_A = S_Ascf.init_guess_by_atom()
Dmat_B = S_Bscf.init_guess_by_atom() #.init_guess_by_minao()

np.trace(Dmat_A), np.trace(Dmat_B)

(8.23741720426768, 14.500372834692605)

# 3) Implement a Freeze and Thaw calculation with the Huzinaga operator

In [15]:
def diagonalize(Sscf, Dold, Sovlp, Fock, n_electrons, tol, n_max):
    
    error = tol * 10
    N = np.zeros((Dold.shape[0]))
    N[n_electrons//2] = 2.    
    i = 0
    while (error > tol) and (i < n_max):
        
        i += 1
        
        if i>1:
            Fock = Sscf.get_hcore() + Sscf.get_veff(dm=Dnew)
            #print(Sscf.get_veff(dm=Dold).shape)
        
        E, C = sp.linalg.eigh(Fock, Sovlp)
        Dnew = sp.dot( C * N, C.transpose().conjugate() )
        
        #Include occupancy of levels

#         norbs = (Sscf.mol.nelectron) // 2
#         e_sorted = np.sort(E)
#         fermi = (e_sorted[norbs] + e_sorted[norbs-1]) / 2.
#         mo_occ = np.zeros_like(E)
#         mo_occ[E<fermi] = 2.
#         Dnew = np.dot(C * mo_occ, C.transpose().conjugate())
        
        error = sp.linalg.norm(Dnew - Dold) / sp.linalg.norm(Dold)
        
    
    
    if i == n_max:
        print("SCF Did Not Converge")
    else:
        print("SCF Converged")
    
    return Dnew, error
    
    


In [16]:
#Obtain the local density matrices using absolute localization and Freeze anad Thaw
convergence_threshold_FT = 1e-2
n_FT_cycles = 100

convergence_threshold_SCF = 1e-2
n_SCF_cycles= 100

i = 0
error = convergence_threshold_FT * 10
while (error > convergence_threshold_FT) and (i < n_FT_cycles):

    i += 1
    
    #System A
    Fock_AA = Fock_T[:na,:na]
    Fock_AB = Fock_T[:na,na:]
    S_AA = M_overlaps[:na,:na]
    S_BA = M_overlaps[na:,:na]
    
    PAA = np.dot(Fock_AB, np.dot( Dmat_B, S_BA ))
    P_huzinaga_A = - 0.5 * ( PAA + PAA.transpose() )
    Fock_AA += P_huzinaga_A
     
    #System B
    Fock_BB = Fock_T[na:,na:]
    Fock_BA = Fock_T[na:,:na]
    S_BB = M_overlaps[na:,na:]
    S_AB = M_overlaps[:na,na:]
    
    PBB = np.dot(Fock_BA, np.dot( Dmat_A, S_AB ))
    P_huzinaga_B = - 0.5 * ( PBB + PBB.transpose() )
    Fock_BB += P_huzinaga_B
    
    
    Dmat_A, err_A = diagonalize(S_Ascf, Dmat_A, S_AA, Fock_AA, S_A.nelectron,
                                convergence_threshold_SCF,n_SCF_cycles)
    Dmat_B, err_B = diagonalize(S_Bscf, Dmat_B, S_BB, Fock_BB, S_B.nelectron,
                                convergence_threshold_SCF,n_SCF_cycles)
                              
    # Updates
    error = err_A + err_B 
    print(err_A,err_B)
    
    if i == n_FT_cycles :
        print("F&T Did Not Converge")


/Users/lex/anaconda3/envs/pyscf/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: scipy.dot is deprecated and will be removed in SciPy 2.0.0, use numpy.dot instead
  app.launch_new_instance()


SCF Did Not Converge
SCF Did Not Converge
0.9539372547684998 5.192282022035384
SCF Did Not Converge
SCF Converged
1.2247448624968236 0.009183780959365347
SCF Did Not Converge
SCF Did Not Converge
1.2247448778662338 0.7707996992630036
SCF Did Not Converge
SCF Did Not Converge
1.2247932304783626 1.1848068680251092
SCF Did Not Converge
SCF Converged
1.2246964819369313 0.008931633809362462
SCF Did Not Converge
SCF Did Not Converge
0.8076271130748077 1.360470499699985
SCF Did Not Converge
SCF Did Not Converge
1.3805344801261865 0.42859494204829585
SCF Did Not Converge
SCF Did Not Converge
1.384920538032085 0.7726701876534929
SCF Did Not Converge
SCF Did Not Converge
1.2247450125774888 1.1843530669494784
SCF Converged
SCF Did Not Converge
0.000701063853670156 0.2914360450612367
SCF Did Not Converge
SCF Converged
1.2250323227293975 0.005188965867440003
SCF Did Not Converge
SCF Did Not Converge
1.2247447589161433 0.2917485597963328
SCF Did Not Converge
SCF Did Not Converge
1.2247449081422108 0

In [17]:
np.trace(Dmat_A), np.trace(Dmat_B)

(2.0000000000000004, 26.80775741707885)

# 4) Perform global fully relaxed KS calculation and account for local correlations

In [18]:
S_Tscf.xc = 'lda,vwn'
S_Tscf.small_rho_cutoff = 1e-20
S_Tscf.kernel()

SCF not converged.
SCF energy = 6145.50457447892


6145.50457447892

In [19]:

S_Ascf_modified = scf.RHF(S_A)

vA = S_Ascf_modified.get_veff(dm=Dmat_A)
Fock_AA = S_Ascf.get_hcore()
Fock_AB = Fock_T[:na,na:]
PAA = np.dot(Fock_AB, np.dot( Dmat_B, S_BA ))
P_huzinaga_A = - 0.5 * ( PAA + PAA.transpose() )
Fock_AA +=  P_huzinaga_A
hcore_modified = Fock_AA - vA


In [20]:
S_Ascf_modified.get_hcore = lambda *args: hcore_modified;
S_Ascf_modified.run();

converged SCF energy = -89.3567318902334


Overwritten attributes  get_hcore  of <class 'pyscf.scf.hf.RHF'>


In [21]:
S_Ascf_modified.e_tot

-89.3567318902334

# 5) Implement Coupled Cluster on $S_A$

In [22]:

S_Acc = cc.CCSD(S_Ascf)
ecc, t1, t2 = S_Acc.kernel()
Ecc = S_Acc.e_tot + ecc

RuntimeError: CCSD Warning: The first argument mf is a DFT object. CCSD calculation should be initialized with HF object.
DFT object can be converted to HF object with the code below:
    mf_hf = scf.RHF(mol)
    mf_hf.__dict__.update(mf_dft.__dict__)


# 6) Obtain final energy $E_{KS} - E_A(DFT) + E_A(CC)$

In [24]:


E_WFinDFT = S_Tscf.e_tot - S_Ascf_modified.e_tot + Ecc

print("WF in DFT energy:", E_WFinDFT)

WF in DFT energy: 6159.23886791259


In [25]:
print("KSDFT energy:", S_Tscf.e_tot)

KSDFT energy: 6145.280472413054


In [26]:
# Perform a FCI to check the validity of results
cisolver = fci.FCI(S_T, S_Tscf.mo_coeff)
efci = cisolver.kernel()[0]
print("FCI energy:", efci)

FCI energy: 6146.03620979643


In [27]:
print("Relative error for WFinDFT:", np.abs(E_WFinDFT-efci)/efci)

Relative error for WFinDFT: 0.0021481582056278206


In [28]:
print("Relative error for KSDFT:", np.abs(S_Tscf.e_tot-efci)/efci)

Relative error for KSDFT: 0.00012296337957974767
